In [ ]:

import sys
import numpy as np
import xarray as xr
import dask.array as da
import numpy.ma as ma
from netCDF4 import Dataset

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as mplcm

import cartopy.crs as ccrs
import matplotlib.ticker as mticker

import cartopy
import cmocean

from decimal import Decimal

%matplotlib inline


In [ ]:
matplotlib.rcParams['figure.figsize']= (20, 8)
matplotlib.rcParams['xtick.labelsize']= 22
matplotlib.rcParams['ytick.labelsize']= 22
matplotlib.rcParams['axes.labelsize']= 22
matplotlib.rcParams.update({'font.size': 22})



In [ ]:
ds=xr.open_dataset('/scratch/cnt0024/hmg2840/albert7a/eNATL60/eNATL60-BLBT02-S/1h/tide/res_harm_ssh_0-360.nc',chunks={'x':1000,'y':1000})
   

In [ ]:
ds_grid=xr.open_dataset('/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/mesh_mask_eNATL60_3.6_lev1.nc4',chunks={'x':1000,'y':1000})

In [ ]:
navlat=ds_grid.nav_lat
navlon=ds_grid.nav_lon
tmask=ds_grid.tmask[0]

In [ ]:
def plot_amp_phase_one_freq(freq):
    amp=ds[str(freq)+'_A']
    amp_mask=ma.masked_where(tmask==0.,amp)
    ph=(180/np.pi)*ds[str(freq)+'_G']
    ph360=da.where(ph<0,ph+360,ph)
    ph360_mask=ma.masked_where(tmask==0.,ph360)    
    cmap=plt.get_cmap('rainbow')
    matplotlib.rcParams['contour.negative_linestyle']= 'solid'
    matplotlib.rcParams['lines.linewidth']= 2

    fig=plt.figure(figsize=(21,18))
    pj= ccrs.PlateCarree()
    ax=plt.subplot(1,1,1,projection=pj)

    ax.coastlines(resolution="10m")
    ax.set_extent([-100,50,0,60])

    gl = ax.gridlines(crs=pj, draw_labels=True, linewidth=1, color='black', alpha=0.2, linestyle='--')
    label_style= {'size' : 20, 'color' : 'black'}
    gl.xlabel_style = label_style
    gl.xlabels_top = False
    gl.xlocator = mticker.FixedLocator(np.arange(-110,100,20,dtype=float))
    gl.ylabels_right = False
    gl.xlabel_style = label_style
    gl.ylocator = mticker.FixedLocator(np.arange(-20,90,10,dtype=float))
    cs = plt.pcolormesh(navlon,navlat,amp_mask,transform=ccrs.PlateCarree(),cmap=cmap,vmin=0,vmax=2)
    plt.contour(navlon,navlat,ph360_mask,[30,60,90,120,150,180,210,240,270,300,330],cmap=plt.get_cmap('tab20b'),linestyle='solid',linewidths=1,transform=ccrs.PlateCarree())
    plt.title(str(freq) +'eNATL60')
    cbar = plt.colorbar(cs,orientation='horizontal',shrink=0.75,label='m')
    plt.savefig('maps_amp_phi_'+str(freq)+'_eNATL60_tidal-tools.png')
    